In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [3]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 18.7 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 


In [21]:
pip install FPDF

  Created wheel for FPDF: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40722 sha256=4c501392dbb31e28837b274eec8e034eb8288ec3f5708d5cc8595bcfa3594b25
  Stored in directory: /root/.cache/pip/wheels/d7/ca/c8/86467e7957bbbcbdf4cf4870fc7dc95e9a16404b2e3c3a98c3
Successfully built FPDF


In [22]:
import pandas as pd
import fpdf
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [5]:
DCdata=pd.read_excel(open('P3_DCLocation.xlsx', 'rb'),
              sheet_name='DCdata')
DCdata.head()

,location,capacity,fixedcost
0,Durham,20655,3831700
1,Fort Wayne,11279,1848400
2,St. Petersburg,25376,2190000
3,Laredo,40169,1361300
4,Buffalo,25167,1312000


In [6]:
dc_loc=DCdata.iloc[:,0].values.tolist()
capacity=DCdata.iloc[:,1].values.tolist()
fixedcost=DCdata.iloc[:,2].values.tolist()

In [7]:
citydata=pd.read_excel(open('P3_DCLocation.xlsx', 'rb'),
              sheet_name='citydata')
citydata.head()

,city,demand
0,New York,8622
1,Los Angeles,3999
2,Chicago,2716
3,Houston,3284
4,Phoenix,1626


In [8]:
city_name=citydata.iloc[:,0].values.tolist()
demand=citydata.iloc[:,1].values.tolist()

In [9]:
distcost=pd.read_excel(open('P3_DCLocation.xlsx', 'rb'),
              sheet_name='distcost')
distcost.head()

,Unnamed: 0,New York,Los Angeles,Chicago,Houston,Phoenix,Philadelphia,San Antonio,San Diego,Dallas,San Jose,Austin,Jacksonville,San Francisco,Columbus,Fort Worth,Indianapolis,Charlotte,Seattle,Denver,"Washington, D.C.",Boston,El Paso,Detroit,Nashville,Memphis,Portland,Oklahoma City,Las Vegas,Louisville,Baltimore,Milwaukee,Albuquerque,Tucson,Fresno,Sacramento,Mesa,Kansas City,Atlanta,Long Beach,Omaha,Raleigh,Colorado Springs,Miami,Virginia Beach,Oakland,Minneapolis,Tulsa,Arlington,New Orleans,Wichita,Cleveland,Tampa,Bakersfield,Aurora,Anaheim,Honolulu,Santa Ana,Riverside,Corpus Christi,Lexington,Stockton,St. Louis,Saint Paul,Henderson,Pittsburgh,Cincinnati,Anchorage,Greensboro,Plano,Newark,Lincoln,Orlando,Irvine,Toledo,Jersey City,Chula Vista
0,Durham,2693,1752,1820,2585,2861,3446,3814,3100,2098,2113,3576,2312,3298,2423,3268,1754,1433,2504,2132,1368,3656,3182,2368,3817,1970,2430,1896,2058,2076,3375,2721,1831,3345,1245,1920,3707,1249,2722,3883,1670,1036,2156,2447,3725,2589,3766,2144,2100,980,2584,1690,3946,3390,2062,3830,3705,2660,2099,1897,1008,3041,1284,1903,1716,2956,3253,2240,2169,2906,1592,3903,1351,3545,2851,1936,3710
1,Fort Wayne,2690,2581,1851,2175,1851,1895,2529,2525,1872,2910,2469,1590,2463,1272,1596,2596,1544,2682,1271,2412,1408,1154,649,1246,1815,2392,3167,2191,1977,1317,2151,280,1561,1749,1776,3064,1004,1710,1150,2697,998,2474,454,1641,1433,553,2226,2290,433,1461,2440,2914,1262,1739,1853,1804,620,2568,2115,2531,2719,2165,2496,1130,2392,440,1042,2098,2770,1614,835,2652,694,294,1702,1341
2,St. Petersburg,2480,3128,2782,2213,2837,3252,2588,2735,1111,2135,3056,851,970,1186,3376,1720,1501,2281,3185,1694,3380,1450,540,764,1966,3232,1937,2442,1223,1858,3261,2936,2887,1422,1380,2915,2784,1776,610,2755,2820,1207,778,2683,528,905,2511,806,906,1393,445,3266,3076,1519,770,2006,3396,1332,820,2843,2052,2392,1697,2508,2957,1509,1521,2142,2392,500,1800,1270,1132,597,1730,3385
3,Laredo,5144,5693,3046,5593,4414,4492,5348,5757,4681,4533,4405,5333,4306,3403,3782,3754,3524,3855,3580,3829,4078,4486,2988,3220,5778,4620,2981,4807,5638,4776,5497,4108,5196,5278,5710,4113,5842,4642,4776,5026,4474,4349,3672,5757,5649,4273,5370,3850,3794,4153,4995,4032,4030,5172,3209,4121,5232,4663,4116,3117,4508,5142,3643,3896,3429,4216,5324,5901,4510,3718,3165,5423,3204,3431,5895,4124
4,Buffalo,3436,4475,4340,5177,4286,4186,4459,4863,5337,5477,4339,4425,3948,5418,4571,5648,5010,4546,5827,3108,4478,4235,5418,5752,5130,5370,5575,2897,4832,4594,4915,3281,3891,3471,5114,4808,5397,3945,4379,5802,3495,3047,4002,5466,4204,5602,4088,4963,5467,5407,4688,5050,3543,3287,5026,3169,5226,3764,5289,3945,4291,3499,3859,5404,3706,4022,5642,5111,5504,4217,4031,4532,5300,5761,4460,4240


In [10]:
dist=distcost.iloc[:,1:].values.tolist()

In [34]:
#Declare a model 
model=ConcreteModel()
num_dc=len(DCdata) #indexed by i
num_city=len(citydata) #indexed by j 
big_m=100000

#Declare DVs
model.x=Var(range(num_dc), domain=Binary)
model.y=Var(range(num_dc),range(num_city),domain=NonNegativeReals)

#Set constraints
#Each DC has a capacity 
model.constraint1=ConstraintList()
for i in range(num_dc): 
   model.constraint1.add(expr= sum(model.y[i,j] for j in range(num_city)) <= capacity[i])

#Demand for each city must be met 
model.constraint2=ConstraintList()
for j in range(num_city):
  model.constraint2.add(expr= sum(model.y[i,j] for i in range(num_dc)) >= demand[j])

#If the DC does not exist, it cannot contribute
model.constraint3=ConstraintList()
for i in range(num_dc):
   model.constraint3.add(expr= sum(model.y[i,j] for j in range(num_city)) <= model.x[i]*big_m)

#Define an objective
model.Objective=Objective(expr=sum(model.x[i]*fixedcost[i]+ model.y[i,j]*dist[i][j] for i in range(num_dc) for j in range(num_city)), sense=minimize)

#Solve
opt=SolverFactory("glpk")
opt.options["mipgap"]=0.001 #tolerance
results=opt.solve(model)

dc_location=[]
for i in range(num_dc):
  if model.x[i]() == 1.0:
    dc_location.append(dc_loc[i])

optimum=model.Objective()

#for i in range(num_dc):
 # for j in range(num_city):
  #    if model.y[i,j]() != 0.0:
   #     print("Distribution Center:", dc_loc[i], "Quantity:", model.y[i,j]() ,"City:", city_name[j])
      
pdf = fpdf.FPDF(format='letter')
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.write(5, "The optimum cost is ")
pdf.write(5, str(model.Objective()))
pdf.ln()

for i in range(num_dc):
  for j in range(num_city):
    if model.y[i,j]() != 0.0:
      pdf.write(5,str(dc_loc[i]))
      pdf.write(5," distribution centre distributes ")
      pdf.write(5,str(model.y[i,j]()))
      pdf.write(5," units to ")
      pdf.write(5,str(city_name[j]))
      pdf.ln()


pdf.output("DClocation&Transportation.pdf")

''